In [ ]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from systems.provided.mike.make_system import make_system

plt.rc('figure', figsize=(30,20))

In [ ]:
system = make_system()
system.config.start_date = datetime.now() - timedelta(days=365*10)
system.config.instruments =  ['AGG', 'BITO', 'EEM', 'FXC', 'FXE', 'FXY', 'GLD', 'HYG', 'IEF', 'LQD', 'PDBC', 'QQQ', 'SHY', 'SPY', 'TLT', 'VYM']

system.config.average_absolute_forecast = 1.0
system.config.forecast_cap = 2.0

system.config.use_forecast_scale_estimates = True
system.config.use_forecast_div_mult_estimates = True


In [ ]:
def plot_all(df, axs, label=None):

    axs = [ax for row in axs for ax in row]
    for i, name in enumerate(df.columns):
        df[name].plot(ax=axs[i], label=label)
        axs[i].set_title(name)
        axs[i].set_xlim(df.index[0], df.index[-1])
        axs[i].legend()

def plot_create_axes(df, n_cols=5, figsize=(30,10)):
    n = df.shape[1]
    n_rows = int(np.ceil(n / n_cols))
    fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols)
    fig.set_size_inches(figsize)
    return fig, axs

instrument_code = 'SPY'
rules = list(system.rules.trading_rules().keys())

In [ ]:
forecasts = pd.concat([system.rules.get_raw_forecast(instrument_code, rule) for rule in rules], axis=1, keys=rules)
fig, axs = plot_create_axes(forecasts, figsize=(30, 10))
plot_all(forecasts, axs)
plt.tight_layout()
plt.show()

In [ ]:
scalars = pd.concat([system.forecastScaleCap.get_forecast_scalar(instrument_code, rule) for rule in rules], axis=1, keys=rules)
fig, axs = plot_create_axes(scalars, figsize=(30,10))
plot_all(scalars, axs)
plt.show()


In [ ]:
raw_forecasts = pd.concat([system.forecastScaleCap.get_raw_forecast(instrument_code, rule) for rule in rules], axis=1, keys=rules)
scaled_forecasts = pd.concat([system.forecastScaleCap.get_scaled_forecast(instrument_code, rule) for rule in rules], axis=1, keys=rules)
capped_forecasts = pd.concat([system.forecastScaleCap.get_capped_forecast(instrument_code, rule) for rule in rules], axis=1, keys=rules)

fig, axs = plot_create_axes(raw_forecasts)
# plot_all(raw_forecasts, axs, label='raw')
plot_all(scaled_forecasts, axs, label='scaled')
plot_all(capped_forecasts, axs, label='capped')

plt.tight_layout()
plt.show()


In [ ]:
all_forecasts = system.combForecast.get_all_forecasts(instrument_code)[rules]
fig, axs = plot_create_axes(all_forecasts)
plot_all(all_forecasts, axs)

In [ ]:
fdm = system.combForecast.get_forecast_diversification_multiplier(instrument_code)
fdm.plot()
plt.xlim(fdm.index[0], fdm.index[-1])

In [ ]:
combinedForecast = system.combForecast.get_combined_forecast(instrument_code)
combinedForecast.plot()